In [143]:
#!/usr/bin/env py27
import warnings
with warnings.catch_warnings():
    warnings.simplefilter('ignore')
from IPython.display import display, Image
import xml.etree.ElementTree as ET
import requests
#import pythoncyc as pc
#from HTMLParser import HTMLParser
#from lxml import html
import Bio
from Bio.Seq import Seq
from Bio.Alphabet import generic_dna
import csv
import pandas as pd
import math
import re
import numpy as np

# this function finds all pathways associated with the molecule of interest. needs work on front end to 
# allow user to define G.O. function associated with molecule.
class organismBreadBoard:
    def __init__(self, name):
        self.name = name
    def pathwayFromCompound(self, compound):
        self.compound = compound
        pathWays = {}
    #request pathway xml from ecocyc server
        resp = requests.get("http://ecocyc.org/apixml?fn=pathways-of-compound&id=ECOLI:{}".format(self.compound))
    #builds element tree from the response xml text file
        root = ET.fromstring(resp.text)
    #pulls all elements associated with Pathway tag
        pathways = root.findall("./Pathway")
    #pulls all pathway IDs
        for pathway in pathways:
            pathID = pathway.attrib['frameid']
            pathWays[pathID] = pathway.findall("common-name")[0].text
        #print (pathWays)
    #return dictionary with pathway ID: common name as key:value pairs
    #print pathWays
        return pathWays

# this function pulls all geneIDs associated with the pathwayID passed into it
    def genesFromPathway(self, pathwayID):
        resp  = requests.get("http://ecocyc.org/apixml?fn=genes-of-pathway&id=ECOLI:{}&detail=full".format(pathwayID))
        root = ET.fromstring(resp.text)
    #print(resp.text)
        genes = root.findall("./Gene")
        genesDict = {}
        for gene in genes:
            geneID = gene.attrib['frameid']
            geneName = gene.findall("common-name")[0].text
            geneStrand= gene.findall("transcription-direction")[0].text
            #print geneStrand
        #print "Gene: " + geneName + ", GeneID: " + geneID
        #page = requests.get("http://biocyc.org/gene?orgid=ECOLI&id=EG11879")
            genesDict[geneID] = [geneName, geneStrand]
        return genesDict
    
# this function pulls the promoterIDs associated with the geneID passed into it
    def getPromoter(self, geneID, strand):
        promoterDict = {}
        resp  = requests.get("http://ecocyc.org/apixml?fn=transcription-units-of-gene&id=ECOLI:{}&detail=full".format(geneID))
        root=ET.fromstring(resp.text)
        #print resp.text
        #transUnits=root.findall("./Transcription-Unit")
        
        promoters=root.findall("./Transcription-Unit/component/Promoter")
        #for transUnit in transUnits:
            #TUID=transUnit.attrib['frameid']
            #print TUID
            #print promoters
        for promoter in promoters:
            #print promoters[promoter]
            pID = promoter.attrib['frameid']
            print pID
            TUID = promoter.find('transcription-unit')
            print TUID
                #promoterDict[pID]=[TUID]

            if pID not in promoterDict:
                try:
                    #absPositPlusOne = promoter.find('absolute-plus-1-pos').text
                    #promoterDict[pID]= absPositPlusOne
                    promoterDict[pID]=[promoter.find('absolute-plus-1-pos').text, strand]
                    #print 'promoter position for:', pID,  'is:', promoterDict[pID]
                #print('b')
                except AttributeError:
                    promoterDict[pID]=['NA','NA']
                    #print 'promoter position for',pID, 'was not found'
            else:
                
                continue
            #promoterDict[pID]= absPositPlusOne
        print promoterDict
        return promoterDict
    
    def getTUID(self, geneID):
        resp  = requests.get("http://ecocyc.org/apixml?fn=transcription-units-of-gene&id=ECOLI:{}&detail=full".format(geneID))
        root=ET.fromstring(resp.text)
        transUnits=root.findall("./Transcription-Unit")
        TU_list=[]
        for transUnit in transUnits:
            TU_list.append(transUnit.attrib['frameid'])
        return TU_list
    
    def getPromoOfTU(self, TUID):
        promoterDict={}
        resp  = requests.get("http://ecocyc.org/apixml?fn=transcription-unit-promoter&id=ECOLI:{}&detail=full".format(TUID))
        root=ET.fromstring(resp.text)
        #print resp.text
        promoters=root.findall("./Promoter")
        #print promoters
        for promoter in promoters:
            pID = promoter.attrib['frameid']
            trnsDir = promoter.findall('transcription-direction')[0].text
            position = promoter.findall('absolute-plus-1-pos')[0].text
            #print pID, trnsDir, position
            promoterDict[pID]=[trnsDir, position]
        return promoterDict
    
    def genesOfTU(self, TU_ID):
        genesList = []
        resp  = requests.get("http://ecocyc.org/apixml?fn=transcription-unit-genes&id=ECOLI:{}&detail=full".format(TU_ID))
        root=ET.fromstring(resp.text)
        print(resp.text)
        #genes = root.findall("./")
        
    def pathwaysOfGene(self, geneID):
        resp  = requests.get("http://ecocyc.org/apixml?fn=pathways-of-gene&id=ECOLI:{}&detail=full".format(geneID))
        root=ET.fromstring(resp.text)
        print(resp.text)
        
    def findAltFunc(self, pathwayID):
        resp  = requests.get("http://metacyc.org/apixml?fn=pathways-of-gene&id=META:{}&detail=full".format(pathwayID))
        root=ET.fromstring(resp.text)
        print(resp.text)
        
    def getGeneSeq(self, geneID):
        resp  = requests.get("http://ecocyc.org/apixml?fn=get-gene-sequence&id=ECOLI:{}&detail=full".format(geneID))
        root=ET.fromstring(resp.text)
        print(resp.text)
        
    def getGenePromoter(self, geneID, strand):
        promoterDict = {}
        resp  = requests.get("http://ecocyc.org/apixml?fn=transcription-units-of-gene&id=ECOLI:{}&detail=full".format(geneID))
        root=ET.fromstring(resp.text)
        #print resp.text
        promoters=root.findall("./Transcription-Unit/component/Promoter")
        for promoter in promoters:
            pID = promoter.attrib['frameid']
            if pID not in promoterDict:
                try:
                    promoterDict[pID]=[promoter.find('absolute-plus-1-pos').text, strand]
                except AttributeError:
                    promoterDict[pID]=['NA','NA']
            else:
                continue
        return promoterDict

In [128]:
class FastAreader :
    '''
    Class to provide reading of a file containing one or more FASTA
    formatted sequences:
    object instantiation:
    FastAreader(<file name>):
 
    object attributes:
    fname: the initial file name
 
    methods:
    readFasta() : returns header and sequence as strings.
    Author: David Bernick
    Date: April 19, 2013
    '''
    def __init__ (self, fname):
        '''contructor: saves attribute fname '''
        self.fname = fname
 
    def readFasta (self):
        '''
        using filename given in init, returns each included FastA record
        as 2 strings - header and sequence.
        whitespace is removed, no adjustment is made to sequence contents.
        The initial '>' is removed from the header.
        '''
        header = ''
        sequence = ''
        
        with open(self.fname) as fileH:
            # initialize return containers
            header = ''
            sequence = ''
 
            # skip to first fasta header
            line = fileH.readline()
            while not line.startswith('>') :
                line = fileH.readline()
            header = line[1:].rstrip()
 
            # header is saved, get the rest of the sequence
            # up until the next header is found
            # then yield the results and wait for the next call.
            # next call will resume at the yield point
            # which is where we have the next header
            for line in fileH:
                if line.startswith ('>'):
                    yield header,sequence
                    header = line[1:].rstrip()
                    sequence = ''
                else :
                    sequence += ''.join(line.rstrip().split()).upper()
        # final header and sequence will be seen with an end of file
        # with clause will terminate, so we do the final yield of the data
        yield header,sequence

In [129]:
#instantiate an organism BB for ecoli
#myOrg=organismBreadBoard('ECOLI')
#myOrg=pc.select_organism('ecoli')
#instantiate a fasta reader with target genome
myReader = FastAreader ('ecoli_v3_u00096_3.fa')
myDFrame1=pd.DataFrame(columns=['Gene','Strand','Sequence','Promoter','Transcription Units','Other genes in operon','Protospacers'])

#parse fastareader and pull out the header and sequence for the genome
targetHead=''
targetSeq=''
for head, seq in myReader.readFasta() :
    targetHead+=head
    targetSeq+= seq
myTable_genes = pd.read_table('All_genes_of_E._coli_K-12_substr._MG1655_10172016_2.txt')
print myTable_genes
for i, row in myTable_genes.iterrows():
    myDFrame1.loc[i,['Gene']]=row['Gene Name']
    myDFrame1.loc[i,['Strand']]=row['Transcription-Direction']
    myDFrame1.loc[i,['Promoter']]=row['Transcription units - promoter of operon']
    myDFrame1.loc[i, ['Transcription Units']]=row['Transcription Units']
    myDFrame1.loc[i, ['Other genes in operon']]=row['Genes of transcription unit']
    myDFrame1.loc[i, ['Sequence']]=row['Sequence - DNA sequence']
    #if not math.isnan(row['Left-End-Position']):
        #geneSeq+=targetSeq[int(row['Left-End-Position']): int(row['Right-End-Position'])]
    #else:
        #continue
    #myDFrame1.loc[i,['Sequence']]=geneSeq
#myDFrame1.to_csv('genes_promos_Ecoli', sep='\t')


     Gene Name Accession-1  Left-End-Position  Right-End-Position  \
0        G7935       b4339          4569358.0           4569918.0   
1        G7934       b4338          4568998.0           4569309.0   
2        G7903       b4282          4506861.0           4507109.0   
3        G7822       b4103          4322443.0           4322811.0   
4        G7798       b3777          3960242.0           3960460.0   
5        G7760       b3443          3583041.0           3583454.0   
6        G7536       b2969          3111128.0           3111988.0   
7        G7478       b2858          2995748.0           2996020.0   
8        G7474       b2854          2994041.0           2994415.0   
9        G7383       b2648          2783065.0           2783208.0   
10       G7222       b2353          2470803.0           2471105.0   
11       G7120       b2083          2167602.0           2167820.0   
12       G7044       b1936          2012502.0           2012780.0   
13       G6632       b1240        

In [130]:
print myDFrame1


          Gene Strand                                           Sequence  \
0        G7935      +  ATGCTATTTTATCACGGTAGCCGTAGTCCTTACCCCTGGTCCCTGT...   
1        G7934      +  ATGACAAACTTCACGACCAGCACGCCGCATGATGCATTATTTAAAA...   
2        G7903      +  TTGCTGGCCGATGAGCTGACCATTGGGCCTATCCGGGCTGTCCCGA...   
3        G7822      -  GTGCTGGCGCTGTTTATCCACACCACCGGCGTGCTCTCCAAGCTGC...   
4        G7798      -  GTGGCGATTAATTTCAGCCCTAAGGTTGGTGAAATACTGGAATGCA...   
5        G7760      +  ATGACTAAAACCCAAATAAATAAATTAATAAAAATGATGAATGATT...   
6        G7536      -  GTGATTCACCAACAGCATATGCGCAACATTGCGCAGTGGTTGCAGG...   
7        G7478      -  ATGCGAAAAAAAATAGAGATGAGCTTGATAAAATCTCCTGCAAATG...   
8        G7474      +  ATGTTCAGAATCGAACCAAATTTAATTAAAGCTATCGCTCTTGTTG...   
9        G7383      -  ATGTGGCATCTTGTCGTGCTGTTGGAGGAGCTGTGCGAACGTGGTA...   
10       G7222      +  ATGCGTCCCGCGACTATCGCCCCATTAACGCCATACGATAAATGGG...   
11       G7120      -  GCCCGTGTAGCAGAGTTCTCCATCAGCCTGGCTACCGGTCGGGCAG...   
12       G70

In [131]:
myTable_promoters = pd.read_table('All_promoters_of_E._coli_K-12_substr._MG1655_10172016.txt')
#print myTable_promoters.head

In [132]:
myTable_promoters = pd.read_table('All_promoters_of_E._coli_K-12_substr._MG1655_10172016.txt')
myDFrame2=pd.DataFrame(columns=['Promoter','Strand','Position','Sequence','Genes in operon','Transcription Units','Pathways'])
upstream = 60
downstream = 60
for i, row in myTable_promoters.iterrows():
    myDFrame2.loc[i,['Promoter']]=row['Promoter Name']
    myDFrame2.loc[i, ['Strand']]=row['Transcription-Direction']
    myDFrame2.loc[i,['Position']]=row['Absolute-Plus-1-Position']
    myDFrame2.loc[i,['Genes in operon']]=row['Promoters - genes downstream of promoter']
    myDFrame2.loc[i, ['Transcription Units']]=row['Promoters - transcription units of promoter']
    myDFrame2.loc[i, ['Genes in operon']]=row['Promoters - genes downstream of promoter']
    myDFrame2.loc[i, ['Pathways']]=row['Transcription units - pathways']
    if not math.isnan(row['Absolute-Plus-1-Position']):
        if row['Absolute-Plus-1-Position']!='-':
            myDFrame2.loc[i, ['Sequence']]=targetSeq[(int(row['Absolute-Plus-1-Position'])-upstream): (int(row['Absolute-Plus-1-Position']+downstream))]
        else:
            myDFrame2.loc[i, ['Sequence']]=targetSeq[(int(row['Absolute-Plus-1-Position'])-downstream): (int(row['Absolute-Plus-1-Position']+upstream))]
    else:
        #print row['Promoter Name']
        #print row['Absolute-Plus-1-Position']
        #myDFrame2.loc[i, ['Sequence']]=targetSeq[(int(row['Absolute-Plus-1-Position'])-55): (int(row['Absolute-Plus-1-Position']+30))]
        continue
print myTable_promoters.head

<bound method DataFrame.head of      Promoter Name  Absolute-Plus-1-Position  \
0        PM0-45785                 3604315.0   
1        PM0-10222                  850209.0   
2        PM0-46187                 1686654.0   
3         PM0-4102                   17317.0   
4         PM0-9365                  498031.0   
5        PM0-44676                  960143.0   
6          PM00083                 1122643.0   
7        PM0-45040                 1764924.0   
8          PM00538                 2798556.0   
9        PM0-45404                 2667928.0   
10       PM0-45769                 3577085.0   
11       PM0-10194                 4478445.0   
12       PM0-46159                 1563201.0   
13        PM0-3664                 1890561.0   
14        PM0-9308                  403687.0   
15       PM0-44660                  922750.0   
16         PM00058                  703708.0   
17       PM0-45024                 1739855.0   
18         PM00522                  953925.0   
19      

In [133]:
print myDFrame2
myDFrame2.to_csv('test_out.CSV')

       Promoter Strand     Position  \
0     PM0-45785      +  3.60432e+06   
1     PM0-10222    NaN       850209   
2     PM0-46187    NaN  1.68665e+06   
3      PM0-4102    NaN        17317   
4      PM0-9365    NaN       498031   
5     PM0-44676      +       960143   
6       PM00083    NaN  1.12264e+06   
7     PM0-45040      -  1.76492e+06   
8       PM00538    NaN  2.79856e+06   
9     PM0-45404      -  2.66793e+06   
10    PM0-45769      -  3.57708e+06   
11    PM0-10194    NaN  4.47844e+06   
12    PM0-46159    NaN   1.5632e+06   
13     PM0-3664    NaN  1.89056e+06   
14     PM0-9308      +       403687   
15    PM0-44660      +       922750   
16      PM00058    NaN       703708   
17    PM0-45024      +  1.73986e+06   
18      PM00522    NaN       953925   
19    PM0-45388      +  2.62967e+06   
20    PM0-45753      +  3.52632e+06   
21    PM0-10177    NaN  4.26428e+06   
22    PM0-46123    NaN  1.34451e+06   
23    PM0-36248    NaN  2.59097e+06   
24     PM0-9061    NaN  4

In [138]:
#Makes protospacers from an arbitrary sequence (in our case either a gene or promoter)

#Note, ignores promoter ids that don't have associated promoter; need to add those.
#Note: Often the algorithm below cannot pull a full protospacer_length protospacer from the given "promoter" region; in this case
#nothing is pulled and an empty comma shows up. Need to update this.

#Currently not adding lists of protospacers directly to dataframe (myDFrame2)-- need to add this capability.

protospacer_length = 30;
PAM_length = 3;
myDFrame2['protospacer-positive'] = ""
myDFrame2['protospacer-negative'] = ""
protospacers = []
for i in range(myDFrame2.shape[0]):
    targetSeq =  myDFrame2.loc[i, ['Sequence']][0]
    protospacers_i = []
    if(type(targetSeq) == str):
        for base in range(len(targetSeq)):
            if (targetSeq[base-2:base] == 'GG') & (base-(protospacer_length+PAM_length)>-1):
                protospacer_positive = targetSeq[base-(protospacer_length+PAM_length):base-PAM_length];
                myDFrame2.loc[i, ['protospacer-positive']] += protospacer_positive + ", " 
                protospacers_i.append(protospacer_positive)
            elif (targetSeq[base-3:base-1] == 'CC') & (base+(protospacer_length)<len(targetSeq)):
                protospacer_negative = targetSeq[base:base+(protospacer_length)];
                myDFrame2.loc[i, ['protospacer-negative']] += protospacer_negative + ", "
                protospacers_i.append(protospacer_negative)
            else:
                continue
    protospacers += protospacers_i
                
myDFrame2.to_csv('CRISPRiTargets.CSV')

In [135]:
#Validation for His -- CRISPathBrick paper/Our paper His protospacer target = tctttaagcggctgcctgcgaattaccgtc
targetSeq =  myDFrame2.loc[91, ['Sequence']][0]
print "Target Sequence: " + targetSeq + "\n"
if(type(targetSeq) == str):
    for base in range(len(targetSeq)):
        if (targetSeq[base-2:base] == 'GG') & (base-(protospacer_length+PAM_length)>-1):
            print 'positive strand'
            print "PAM: " + targetSeq[base-PAM_length:base]
            print "Protospacer: " + targetSeq[base-(protospacer_length+PAM_length):base-PAM_length] + "\n"
        elif (targetSeq[base-3:base-1] == 'CC') & (base+(protospacer_length)<len(targetSeq)):
            print 'negative strand'
            print "PAM: " + targetSeq[base-PAM_length:base]
            print "Protospacer: " + targetSeq[base:base+(protospacer_length)] + "\n"
        else:
            continue

Target Sequence: GCGCGCTTTAAAGCCTCCAGTGCGGTGTTTAAATCTTTGTGGGATCAGGGCATTATCTTACGTGATCAGAATAAACAACCCTCTTTAAGCGGCTGCCTGCGAATTACCGTCGGAACCCGT

negative strand
PAM: CCT
Protospacer: CCAGTGCGGTGTTTAAATCTTTGTGGGATC

negative strand
PAM: CCA
Protospacer: GTGCGGTGTTTAAATCTTTGTGGGATCAGG

positive strand
PAM: TGG
Protospacer: AAAGCCTCCAGTGCGGTGTTTAAATCTTTG

positive strand
PAM: GGG
Protospacer: AAGCCTCCAGTGCGGTGTTTAAATCTTTGT

positive strand
PAM: AGG
Protospacer: CCAGTGCGGTGTTTAAATCTTTGTGGGATC

positive strand
PAM: GGG
Protospacer: CAGTGCGGTGTTTAAATCTTTGTGGGATCA

negative strand
PAM: CCC
Protospacer: TCTTTAAGCGGCTGCCTGCGAATTACCGTC

negative strand
PAM: CCT
Protospacer: CTTTAAGCGGCTGCCTGCGAATTACCGTCG

positive strand
PAM: CGG
Protospacer: ACGTGATCAGAATAAACAACCCTCTTTAAG

positive strand
PAM: CGG
Protospacer: CTCTTTAAGCGGCTGCCTGCGAATTACCGT



In [137]:
## Important numbers:

numPromoters = myDFrame2.shape[0]

print "Number of promoters annotated in E. coli: " + str(numPromoters)
print "Average number of protospacers identified per promoter (searching from " + str(upstream) + " bps upstream and " + str(downstream) + " bps downstream): " + str(len(protospacers)/numPromoters)

Number of promoters annotated in E. coli: 3841
Average number of protospacers identified per promoter (searching from 60 bps upstream and 60 bps downstream): 7
